# SensRay Ray Tracing & Sensitivity Kernels Demo

This notebook demonstrates:
- Ray tracing with ObsPy/TauP integration
- Computing per-cell ray path lengths
- Calculating sensitivity kernels
- Visualizing rays and kernels in cross-sections

In [1]:
import os
# Top-level INTERACTIVE switch: set to True for interactive Jupyter widgets,
# or False to force static/off-screen rendering and enable screenshot-based outputs.
INTERACTIVE = False

# If not interactive, set PyVista to off-screen mode. If interactive, prefer ipyvtklink/panel.
if not INTERACTIVE:
    os.environ['PYVISTA_OFF_SCREEN'] = 'true'
    os.environ['PYVISTA_USE_IPYVTK'] = 'false'
else:
    os.environ.pop('PYVISTA_OFF_SCREEN', None)
    # Prefer ipyvtklink for Jupyter interactive 3D plots (requires ipyvtklink installed)
    os.environ['PYVISTA_USE_IPYVTK'] = 'true'


# Configure PyVista backend according to INTERACTIVE flag
try:
    import pyvista as pv
    if INTERACTIVE:
        try:
            pv.set_jupyter_backend('ipyvtklink')
            print("Configured PyVista for interactive plotting using ipyvtklink")
        except Exception:
            try:
                pv.set_jupyter_backend('panel')
                print("Configured PyVista for interactive plotting using panel")
            except Exception as e:
                print(f"Warning: Could not set interactive PyVista backend: {e}")
    else:
        pv.set_jupyter_backend('static')
        print("Configured PyVista for static/off-screen plotting")
except Exception as e:
    print(f"Warning: Could not import/configure pyvista: {e}")

Configured PyVista for static/off-screen plotting


## 1. Setup Model and Mesh

Create a model and mesh for ray tracing experiments.

### Creating a layered tetrahedral mesh (discontinuities)

This demo uses a layered, concentric-sphere tetrahedral mesh so you can control resolution across major internal interfaces (discontinuities). The mesher accepts these main controls:

- `radii` (list of floats, ascending): interface radii in km (last entry must be the outer radius).
- `H_layers` (list of floats): target element size per layer (km). If `None`, `mesh_size_km` is used for all layers.
- `W_trans` (list of floats): half-widths for smooth size transitions at interfaces (km). If omitted, a default ~0.2*layer_thickness is used.

Notes:
- Units are kilometres throughout the API.
- For a uniform sphere just omit `radii` and set `mesh_size_km`.

Example (run this when creating a new mesh):
```python
radii = [1221.5, 3480.0, 6371.0]
H_layers = [500.0, 500.0, 300.0]
W_trans = [50.0, 100.0]  # optional
# Create mesh (use do_optimize=False for faster development)
model.create_mesh(mesh_size_km=1000.0, radii=radii, H_layers=H_layers, W_trans=W_trans, do_optimize=False)
model.mesh.populate_properties(['vp', 'vs', 'rho'])
model.mesh.save('prem_mesh.vtu')  # file extension recommended
```

In [2]:
import numpy as np
from scipy.sparse import csr_matrix
from sensray import CoordinateConverter, PlanetModel

# Load model and create mesh
model = PlanetModel.from_standard_model('prem')
# Create mesh and save if not exist, otherwise load existing
mesh_path = "prem_mesh"
try:
    model.create_mesh(from_file=mesh_path)
    print(f"Loaded existing mesh from {mesh_path}")
except FileNotFoundError:
    print("Creating new mesh...")
    radii = [1221.5, 3480.0, 6371]
    H_layers = [1000, 1000, 600]
    model.create_mesh(mesh_size_km=1000, radii=radii, H_layers=H_layers)
    model.mesh.populate_properties(['vp', 'vs', 'rho'])
    model.mesh.save("prem_mesh")  # Save mesh to VT
print(f"Created mesh: {model.mesh.mesh.n_cells} cells")

Loaded mesh from prem_mesh.vtu
Loaded metadata: 18337 cells, 3988 points
Loaded existing mesh from prem_mesh
Created mesh: 18337 cells


## 2. Define Source-Receiver Geometry

Generate random earthquake-receiver positions with a function, or define manually.

In [3]:
# function to generate random points
def point(pointType="Source", minLat=-90, maxLat=90, minLon=-180, maxLon=180, minDepth=0, maxDepth=700):
    if pointType == "Source":
        lat = np.random.uniform(minLat, maxLat)
        lon = np.random.uniform(minLon, maxLon)
        depth = np.random.uniform(minDepth, maxDepth)  # depth in km
        return (lat, lon, depth)
    elif pointType == "Receiver":
        lat = np.random.uniform(minLat, maxLat)
        lon = np.random.uniform(minLon, maxLon)
        return (lat, lon)
    else:
        raise ValueError("pointType must be 'Source' or 'Receiver'")


phases = ["P", "S", "ScS"]

# # Generate source and receiver points and combinations
# sources = [point("Source", minDepth=150, maxDepth=150) for _ in range(2)]
# receivers = [point("Receiver", maxDepth=0) for _ in range(5)]
# srp = [prod + tuple([phases]) for prod in product(sources, receivers)]

# testing with one source-receiver pair - same as initial test
source_lat, source_lon, source_depth = 0.0, 0.0, 150.0  # Equator, 150 km depth
receiver_lat, receiver_lon = 30.0, 45.0  # Surface station
srp = [((source_lat, source_lon, source_depth), (receiver_lat, receiver_lon), ["P", "S", "ScS"])]



## 3. Ray Tracing with TauP and Travel Time calculation

Class creation of forward operator G with parameters of model, and list of (source, receiver, phases). 
Computes ray paths for different seismic phases of interest for each source-receiver pair and add to kernel matrix. Make sparse matrix and dot with the model to compute the travel times for the phases of interest. 

In [4]:
class G:
    def __init__(self, model, srp):
        self.__model = model
        self.__srp = srp  # list of source, rec, phases of interest

        # set up kernel matrix dict
        self.__kernel_matrices = {}

        # Calculate kernel matrix
        self.__calcMatrix__()
        
    def __calcMatrix__(self):
        # calculate kernels and add to dense matrix
        for (source_lat, source_lon, source_depth), (receiver_lat, receiver_lon), phases in self.__srp:
            self.__computeKernels__(self.__model, source_lat, source_lon, source_depth, receiver_lat, receiver_lon, phases)

        # transform dense matrices to sparse
        for phase, kernel_mat in self.__kernel_matrices.items():
            self.__kernel_matrices[phase] = csr_matrix(np.array(kernel_mat))
            print(f"{phase} Kernel Matrix shape: {self.__kernel_matrices[phase].shape}, nnz: {self.__kernel_matrices[phase].nnz}")

    def __computeKernels__(self, model, source_lat, source_lon, source_depth, receiver_lat, receiver_lon, phases):
        print(f"Source: ({source_lat}°, {source_lon}°, {source_depth} km)")
        print(f"Receiver: ({receiver_lat}°, {receiver_lon}°, 0 km)")
        print(f"Phases: {phases}")
        
        # Compute great-circle plane normal for cross-sections
        plane_normal = CoordinateConverter.compute_gc_plane_normal(
            source_lat, source_lon, receiver_lat, receiver_lon
        )

        # Get ray paths for P and S waves
        rays = model.taupy_model.get_ray_paths_geo(
            source_depth_in_km=source_depth,
            source_latitude_in_deg=source_lat,
            source_longitude_in_deg=source_lon,
            receiver_latitude_in_deg=receiver_lat,
            receiver_longitude_in_deg=receiver_lon,
            phase_list=phases
        )

        # kernel calculation
        print(f"Found {len(rays)} ray paths:")
        for i, ray in enumerate(rays):
            print(f"  {i+1}. {ray.phase.name}: {ray.time:.2f} s, {len(ray.path)} points")
            length = model.mesh.add_ray_to_mesh(ray, f"{ray.phase.name}_wave")
            kernel = model.mesh.compute_sensitivity_kernel(
                ray, property_name=f'v{ray.phase.name[0].lower()}', attach_name=f'K_{ray.phase.name}_v{ray.phase.name[0].lower()}', epsilon=1e-6
            )
            # Adds a new entry if phase not already in dict
            if ray.phase.name not in self.__kernel_matrices:
                self.__kernel_matrices[ray.phase.name] = []
            self.__kernel_matrices[ray.phase.name].append(kernel)


    def __apply__(self, model=None):
        # compute travel times from kernels
        print("Computing travel times from kernels...")
        times = {}
        for phase, kernel_matrix in self.__kernel_matrices.items():
            if kernel_matrix.shape[0] > 0:
                # use provided model or default
                model = self.__model if model is None else model
                times[phase] = kernel_matrix.dot(model.mesh.mesh.cell_data['v' + phase[0].lower()])
                print(f"{phase} travel times: min {times[phase].min():.2f} s, max {times[phase].max():.2f} s")

        return times

appl = G(model, srp)
travel_times = appl.__apply__(model)
print(travel_times)

Source: (0.0°, 0.0°, 150.0 km)
Receiver: (30.0°, 45.0°, 0 km)
Phases: ['P', 'S', 'ScS']


Building obspy.taup model for '/home/matth/miniconda3/envs/masters-project/lib/python3.11/site-packages/sensray/models/prem.nd' ...
filename = /home/matth/miniconda3/envs/masters-project/lib/python3.11/site-packages/sensray/models/prem.nd
Done reading velocity model.
Radius of model . is 6371.0
Using parameters provided in TauP_config.ini (or defaults if not) to call SlownessModel...
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 kilometers
taup.create.max_range_interval = 0.04363323129985824 degrees
taup.create.max_interp_error = 0.05 seconds
taup.create.allow_inner_core_s = True
Slow model  553 P layers,646 S layers
Done calculating Tau branches.
Done Saving /tmp/prem.npz
Method run is done, but not necessarily successful.
Found 3 ray paths:
  1. P: 535.08 s, 227 points
Stored ray path lengths as cell data: 'ray_P_wave_P_lengths'
Stored sensitivity kernel as cell data: 'K_P_vp'
  2. S: 969.

## 4. Export Results

Save mesh with all computed properties for further analysis.

In [5]:
# Save mesh with rays and kernels
model.mesh.save('prem_mesh_with_rays_kernels')

# Show what was saved
info = model.mesh.list_properties(show_stats=False)
print(f"Saved {len(info['cell_data'])} properties to VTU file:")
for prop in info['cell_data'].keys():
    print(f"  - {prop}")

print("\nFiles created:")
print("  - prem_mesh_with_rays_kernels.vtu (mesh + all data)")
print("  - prem_mesh_with_rays_kernels_metadata.json (property list)")

Saved mesh to prem_mesh_with_rays_kernels.vtu
Saved metadata to prem_mesh_with_rays_kernels_metadata.json
Mesh properties summary:
  cell_data keys: ['gmsh:physical', 'gmsh:geometrical', 'layer_0-layer_1-layer_2-gmsh:bounding_entities', 'region', 'vp', 'vs', 'rho', 'ray_P_wave_P_lengths', 'K_P_vp', 'ray_S_wave_S_lengths', 'K_S_vs', 'ray_ScS_wave_ScS_lengths', 'K_ScS_vs']
Saved 13 properties to VTU file:
  - gmsh:physical
  - gmsh:geometrical
  - layer_0-layer_1-layer_2-gmsh:bounding_entities
  - region
  - vp
  - vs
  - rho
  - ray_P_wave_P_lengths
  - K_P_vp
  - ray_S_wave_S_lengths
  - K_S_vs
  - ray_ScS_wave_ScS_lengths
  - K_ScS_vs

Files created:
  - prem_mesh_with_rays_kernels.vtu (mesh + all data)
  - prem_mesh_with_rays_kernels_metadata.json (property list)
